In [112]:
import pandas as pd
import numpy as np
import glob
import os
from datetime import datetime, date
import holidays

In [113]:
# load zonal demand data 
demand_files = glob.glob('../HourlyZonalDemand/PUB_DemandZonal_*.csv')
dfs = []

for file in demand_files:
    df = pd.read_csv(file, skiprows=3)  # Skip header rows
    dfs.append(df)

hourly_demand = pd.concat(dfs, ignore_index=True)

# Convert date and hour to datetime
hourly_demand['datetime'] = pd.to_datetime(hourly_demand['Date']) + pd.to_timedelta(hourly_demand['Hour'] - 1, unit='h')
hourly_demand = hourly_demand.sort_values('datetime')

# Filter hourly_demand to only include data until 2025-04-04
hourly_demand = hourly_demand[hourly_demand['datetime'] < '2025-04-05']

# drop unrequired columns
hourly_demand = hourly_demand.drop(columns=['Diff', 'Zone Total', "Date", "Hour"]) # we add hour back in again later for consistency  

display(hourly_demand)




,Ontario Demand,Northwest,Northeast,Ottawa,East,Toronto,Essa,Bruce,Southwest,Niagara,West,datetime
0,16627,591,1466,1066,1268,5340,1579,89,3157,477,1489,2018-01-01 00:00:00
1,16084,577,1420,985,1300,5211,1516,91,3061,462,1465,2018-01-01 01:00:00
2,15866,613,1417,943,1316,5096,1471,86,3032,446,1441,2018-01-01 02:00:00
3,15725,656,1430,943,1303,4987,1451,81,2911,439,1413,2018-01-01 03:00:00
4,15470,657,1425,932,1343,4926,1422,69,2907,449,1391,2018-01-01 04:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...
63619,16611,641,1360,1007,1122,6004,1150,119,3139,565,1754,2025-04-04 19:00:00
63620,16209,633,1381,991,1058,5859,1120,113,2995,553,1698,2025-04-04 20:00:00
63621,15299,624,1363,954,1009,5537,1052,111,2842,521,1556,2025-04-04 21:00:00
63622,14235,613,1316,876,945,5198,972,108,2665,511,1447,2025-04-04 22:00:00


In [114]:
# hourly ontario demand 

hourly_ontario_demand = hourly_demand[["datetime", "Ontario Demand"]]
hourly_ontario_demand.set_index(['datetime'], inplace=True)
display(hourly_ontario_demand)

,Ontario Demand
datetime,
2018-01-01 00:00:00,16627
2018-01-01 01:00:00,16084
2018-01-01 02:00:00,15866
2018-01-01 03:00:00,15725
2018-01-01 04:00:00,15470
...,...
2025-04-04 19:00:00,16611
2025-04-04 20:00:00,16209
2025-04-04 21:00:00,15299


In [115]:
# load hourly climate data from different regions into a multi-level dataframe
climate_files = glob.glob('../ClimateData/*Climate.csv')
dfs = []

for file in climate_files:
    # Extract region name from filename (e.g., "NortheastClimate.csv" -> "Northeast")
    region = os.path.basename(file).split('Climate')[0]
    df = pd.read_csv(file, skiprows=3, nrows=63600) 
    # there are a bunch of missing values after 63604, i.e. April 5th data is incomplete so we shouldn't use it
    # data goes until line 63676 after that we have mean data for each day
    df['datetime'] = pd.to_datetime(df['time'])
    df['region'] = region
    df = df.drop(columns=["time"])
    
    # Set multi-index with datetime and region
    df.set_index(['datetime', 'region'], inplace=True)
    dfs.append(df)

# Combine all climate dataframes
hourly_climate = pd.concat(dfs)

# Sort the multi-index
hourly_climate.sort_index(inplace=True)

display(hourly_climate)

temperature_2m (°C)  cloud_cover (%)  \
datetime            region                                            
2018-01-02 00:00:00 Bruce                     -6.8               98   
                    East                     -17.6               25   
                    Essa                     -22.6                4   
                    Niagara                  -12.4              100   
                    Northeast                -24.2              100   
...                                            ...              ...   
2025-04-04 23:00:00 Northwest                  0.1              100   
                    Ottawa                     1.9              100   
                    Southwest                  5.2              100   
                    Toronto                    5.0              100   
                    West                       5.4              100   

                               cloud_cover_low (%)  cloud_cover_mid (%)  \
datetime            region                                                
2018-01-02 00:00:00 Bruce                       96                   98   
                    East                        25                    0   
                    Essa                         4                    0   
                    Niagara                    100                  100   
                    Northeast                    0                  100   
...                                            ...                  ...   
2025-04-04 23:00:00 Northwest                   18                   99   
                    Ottawa                       0                    0   
                    Southwest                    0                   88   
                    Toronto                      0                  100   
                    West                         0                   57   

                               cloud_cover_high (%)  wind_speed_10m (km/h)  \
datetime            region                                                   
2018-01-02 00:00:00 Bruce                         0                   32.2   
                    East                          0                   15.9   
                    Essa                          0                    5.9   
                    Niagara                       0                   11.7   
                    Northeast                   100                   11.5   
...                                             ...                    ...   
2025-04-04 23:00:00 Northwest                    47                   12.4   
                    Ottawa                      100                    7.4   
                    Southwest                   100                   14.8   
                    Toronto                     100                    7.2   
                    West                        100                   19.1   

                               wind_speed_100m (km/h)  \
datetime            region                              
2018-01-02 00:00:00 Bruce                        47.4   
                    East                         26.5   
                    Essa                         12.5   
                    Niagara                      22.7   
                    Northeast                    25.5   
...                                               ...   
2025-04-04 23:00:00 Northwest                    24.6   
                    Ottawa                       15.0   
                    Southwest                    27.5   
                    Toronto                      17.6   
                    West                         35.8   

                               weather_code (wmo code)  precipitation (mm)  \
datetime            region                                                   
2018-01-02 00:00:00 Bruce                           71                 0.2   
                    East                             1                 0.0   
                    Essa                             0        

In [116]:
# combine hourly demand and climate data

# melt demand data, i.e. make it such that each row is a unique datetime and region combination 
merge_hourly_demand = hourly_demand.melt(
    id_vars=['datetime'],
    value_vars=['Northwest', 'Northeast', 'Ottawa', 'East', 'Toronto', 'Essa', 'Bruce', 'Southwest', 'Niagara', 'West'],
    var_name='region',
    value_name='zonal_demand'
)

# merge 
hourly_data = pd.merge(
    hourly_climate,
    merge_hourly_demand[['datetime', 'region', 'zonal_demand']],
    on=['datetime', 'region'],
    how='left'  # Use 'inner' if you only want matching rows
)
hourly_data.set_index(['datetime', 'region'], inplace=True)

display(hourly_data)



temperature_2m (°C)  cloud_cover (%)  \
datetime            region                                            
2018-01-02 00:00:00 Bruce                     -6.8               98   
                    East                     -17.6               25   
                    Essa                     -22.6                4   
                    Niagara                  -12.4              100   
                    Northeast                -24.2              100   
...                                            ...              ...   
2025-04-04 23:00:00 Northwest                  0.1              100   
                    Ottawa                     1.9              100   
                    Southwest                  5.2              100   
                    Toronto                    5.0              100   
                    West                       5.4              100   

                               cloud_cover_low (%)  cloud_cover_mid (%)  \
datetime            region                                                
2018-01-02 00:00:00 Bruce                       96                   98   
                    East                        25                    0   
                    Essa                         4                    0   
                    Niagara                    100                  100   
                    Northeast                    0                  100   
...                                            ...                  ...   
2025-04-04 23:00:00 Northwest                   18                   99   
                    Ottawa                       0                    0   
                    Southwest                    0                   88   
                    Toronto                      0                  100   
                    West                         0                   57   

                               cloud_cover_high (%)  wind_speed_10m (km/h)  \
datetime            region                                                   
2018-01-02 00:00:00 Bruce                         0                   32.2   
                    East                          0                   15.9   
                    Essa                          0                    5.9   
                    Niagara                       0                   11.7   
                    Northeast                   100                   11.5   
...                                             ...                    ...   
2025-04-04 23:00:00 Northwest                    47                   12.4   
                    Ottawa                      100                    7.4   
                    Southwest                   100                   14.8   
                    Toronto                     100                    7.2   
                    West                        100                   19.1   

                               wind_speed_100m (km/h)  \
datetime            region                              
2018-01-02 00:00:00 Bruce                        47.4   
                    East                         26.5   
                    Essa                         12.5   
                    Niagara                      22.7   
                    Northeast                    25.5   
...                                               ...   
2025-04-04 23:00:00 Northwest                    24.6   
                    Ottawa                       15.0   
                    Southwest                    27.5   
                    Toronto                      17.6   
                    West                         35.8   

                               weather_code (wmo code)  precipitation (mm)  \
datetime            region                                                   
2018-01-02 00:00:00 Bruce                           71                 0.2   
                    East                             1                 0.0   
                    Essa                             0        

In [117]:
# get daily climate data 

climate_files = glob.glob('../ClimateData/*Climate.csv')
dfs = []

for file in climate_files:
    # Extract region name from filename (e.g., "NortheastClimate.csv" -> "Northeast")
    region = os.path.basename(file).split('Climate')[0]
    df = pd.read_csv(file, skiprows=63677) 
    df = df.head(-3)
    # need to read rows 63678-66328 
    df['date'] = pd.to_datetime(df['time'])
    df['region'] = region
    # latter two are rows with NaN values
    df = df.drop(columns=["time", "precipitation_probability_mean (undefined)", "precipitation_probability_min (undefined)"])
    df['day_length'] = pd.to_datetime(df['sunset (iso8601)']) - pd.to_datetime(df['sunrise (iso8601)'])
    # Set multi-index with datetime and region
    df.set_index(['date', 'region'], inplace=True)
    dfs.append(df)

# Combine all climate dataframes
daily_climate = pd.concat(dfs)

# Sort the multi-index
daily_climate.sort_index(inplace=True)

display(daily_climate)

temperature_2m_mean (°C)  cloud_cover_mean (%)  \
date       region                                                      
2018-01-02 Bruce                          -6.7                  98.0   
           East                          -13.4                  76.0   
           Essa                          -12.6                  94.0   
           Niagara                       -10.5                  76.0   
           Northeast                     -16.3                  94.0   
...                                        ...                   ...   
2025-04-04 Northwest                       0.2                  82.0   
           Ottawa                          4.9                  49.0   
           Southwest                       5.1                  71.0   
           Toronto                         5.3                  67.0   
           West                            7.3                  82.0   

                      cloud_cover_max (%)  cloud_cover_min (%)  \
date       region                                                
2018-01-02 Bruce                    100.0                 86.0   
           East                     100.0                  0.0   
           Essa                     100.0                  4.0   
           Niagara                  100.0                  1.0   
           Northeast                100.0                 37.0   
...                                   ...                  ...   
2025-04-04 Northwest                100.0                  0.0   
           Ottawa                   100.0                  0.0   
           Southwest                100.0                  0.0   
           Toronto                  100.0                  0.0   
           West                     100.0                  0.0   

                      wind_speed_10m_mean (km/h)  temperature_2m_min (°C)  \
date       region                                                           
2018-01-02 Bruce                            33.8                     -8.3   
           East                             17.0                    -22.9   
           Essa                             14.9                    -22.6   
           Niagara                          21.3                    -14.5   
           Northeast                        12.5                    -24.2   
...                                          ...                      ...   
2025-04-04 Northwest                        11.1                     -4.1   
           Ottawa                           11.7                     -0.5   
           Southwest                         8.3                     -0.8   
           Toronto                           7.1                     -0.3   
           West                             10.6                      1.9   

                      apparent_temperature_mean (°C)  weather_code (wmo code)  \
date       region                                                               
2018-01-02 Bruce                               -14.9                     71.0   
           East                                -19.5                      3.0   
           Essa                                -18.3                     73.0   
           Niagara                             -17.1                     73.0   
           Northeast                           -22.0                     71.0   
...                                              ...                      ...   
2025-04-04 Northwest                            -4.2                     51.0   
           Ottawa                                1.1                      3.0   
           Southwest                             2.1                      3.0   
           Toronto                               2.5                      3.0   
           West                                  3.9                     51.0   

                      temperature_2m_max (°C)  apparent_temperature_max (°C)  \
date       region                                                              
2018-01-02

In [118]:
# daily ontario demand 

# Assuming df contains your data
daily_demand = hourly_demand
daily_demand['date'] = pd.to_datetime(daily_demand['datetime'].dt.date)
daily_demand = daily_demand.drop(columns=["datetime"])

# Select only numeric columns to sum 
numeric_columns = hourly_demand.select_dtypes(include=['number']).columns

daily_demand = daily_demand.groupby('date')[numeric_columns].sum().reset_index()
daily_ontario_demand = daily_demand[["date", "Ontario Demand"]]
daily_ontario_demand.set_index(['date'], inplace=True)
daily_demand = daily_demand.drop(columns=["Ontario Demand"])
display(daily_ontario_demand)
display(daily_demand)

,Ontario Demand
date,
2018-01-01,406340
2018-01-02,430873
2018-01-03,434960
2018-01-04,433198
2018-01-05,455055
...,...
2025-03-31,373406
2025-04-01,375761
2025-04-02,406097


,date,Northwest,Northeast,Ottawa,East,Toronto,Essa,Bruce,Southwest,Niagara,West
0,2018-01-01,15249,34493,26274,34951,134028,36774,1988,77389,11439,36150
1,2018-01-02,14910,33191,26816,35831,147904,36823,1984,84959,12378,38792
2,2018-01-03,15040,34490,25393,32411,149297,36256,3111,86718,12628,41264
3,2018-01-04,15221,35047,25343,31886,150517,37329,3214,85344,12795,39718
4,2018-01-05,15164,36078,31048,32305,157676,40589,2556,87574,13392,40063
...,...,...,...,...,...,...,...,...,...,...,...
2646,2025-03-31,13447,34069,25221,21073,134218,21408,2398,70930,12661,40556
2647,2025-04-01,13220,32852,24542,17732,137583,23485,2725,72793,12767,40585
2648,2025-04-02,13929,33213,25172,23618,147589,27506,2505,78807,13582,42160
2649,2025-04-03,13792,32721,23833,21347,134509,24095,2310,70459,12819,37162


In [119]:
# combine daily demand and climate data

# melt demand data, i.e. make it such that each row is a unique datetime and region combination 
merge_daily_demand = daily_demand.melt(
    id_vars=['date'],
    value_vars=['Northwest', 'Northeast', 'Ottawa', 'East', 'Toronto', 'Essa', 'Bruce', 'Southwest', 'Niagara', 'West'],
    var_name='region',
    value_name='zonal_demand'
)

# merge 
daily_data = pd.merge(
    daily_climate,
    merge_daily_demand[['date', 'region', 'zonal_demand']],
    on=['date', 'region'],
    how='left'  # Use 'inner' if you only want matching rows
)
daily_data.set_index(['date', 'region'], inplace=True)

display(daily_data)

temperature_2m_mean (°C)  cloud_cover_mean (%)  \
date       region                                                      
2018-01-02 Bruce                          -6.7                  98.0   
           East                          -13.4                  76.0   
           Essa                          -12.6                  94.0   
           Niagara                       -10.5                  76.0   
           Northeast                     -16.3                  94.0   
...                                        ...                   ...   
2025-04-04 Northwest                       0.2                  82.0   
           Ottawa                          4.9                  49.0   
           Southwest                       5.1                  71.0   
           Toronto                         5.3                  67.0   
           West                            7.3                  82.0   

                      cloud_cover_max (%)  cloud_cover_min (%)  \
date       region                                                
2018-01-02 Bruce                    100.0                 86.0   
           East                     100.0                  0.0   
           Essa                     100.0                  4.0   
           Niagara                  100.0                  1.0   
           Northeast                100.0                 37.0   
...                                   ...                  ...   
2025-04-04 Northwest                100.0                  0.0   
           Ottawa                   100.0                  0.0   
           Southwest                100.0                  0.0   
           Toronto                  100.0                  0.0   
           West                     100.0                  0.0   

                      wind_speed_10m_mean (km/h)  temperature_2m_min (°C)  \
date       region                                                           
2018-01-02 Bruce                            33.8                     -8.3   
           East                             17.0                    -22.9   
           Essa                             14.9                    -22.6   
           Niagara                          21.3                    -14.5   
           Northeast                        12.5                    -24.2   
...                                          ...                      ...   
2025-04-04 Northwest                        11.1                     -4.1   
           Ottawa                           11.7                     -0.5   
           Southwest                         8.3                     -0.8   
           Toronto                           7.1                     -0.3   
           West                             10.6                      1.9   

                      apparent_temperature_mean (°C)  weather_code (wmo code)  \
date       region                                                               
2018-01-02 Bruce                               -14.9                     71.0   
           East                                -19.5                      3.0   
           Essa                                -18.3                     73.0   
           Niagara                             -17.1                     73.0   
           Northeast                           -22.0                     71.0   
...                                              ...                      ...   
2025-04-04 Northwest                            -4.2                     51.0   
           Ottawa                                1.1                      3.0   
           Southwest                             2.1                      3.0   
           Toronto                               2.5                      3.0   
           West                                  3.9                     51.0   

                      temperature_2m_max (°C)  apparent_temperature_max (°C)  \
date       region                                                              
2018-01-02

In [120]:
datasets = [hourly_data, daily_data, hourly_ontario_demand, daily_ontario_demand]

for df in datasets:
    print(df.index.get_level_values(0))
# hourly_data["datetime"]
# hourly_ontario_demand["datetime"]

DatetimeIndex(['2018-01-02 00:00:00', '2018-01-02 00:00:00',
               '2018-01-02 00:00:00', '2018-01-02 00:00:00',
               '2018-01-02 00:00:00', '2018-01-02 00:00:00',
               '2018-01-02 00:00:00', '2018-01-02 00:00:00',
               '2018-01-02 00:00:00', '2018-01-02 00:00:00',
               ...
               '2025-04-04 23:00:00', '2025-04-04 23:00:00',
               '2025-04-04 23:00:00', '2025-04-04 23:00:00',
               '2025-04-04 23:00:00', '2025-04-04 23:00:00',
               '2025-04-04 23:00:00', '2025-04-04 23:00:00',
               '2025-04-04 23:00:00', '2025-04-04 23:00:00'],
              dtype='datetime64[ns]', name='datetime', length=636000, freq=None)
DatetimeIndex(['2018-01-02', '2018-01-02', '2018-01-02', '2018-01-02',
               '2018-01-02', '2018-01-02', '2018-01-02', '2018-01-02',
               '2018-01-02', '2018-01-02',
               ...
               '2025-04-04', '2025-04-04', '2025-04-04', '2025-04-04',
              

In [121]:
# we need to modify this so that it works for the multi-index merged dataframe, 
# but also perhaps preserve this so that we can use it for the other dataframe that has the daily demand data 

def add_time_features(df):
    datetime_index = df.index.get_level_values(0)
    
    # df['day_of_week'] = df['datetime'].dt.dayofweek
    # df['month'] = df['datetime'].dt.month
    # df['day_of_year'] = df['datetime'].dt.dayofyear
    
    df['day_of_week'] = datetime_index.dayofweek
    df['month'] = datetime_index.month
    df['day_of_year'] = datetime_index.dayofyear
    
    # Add holiday information
    holiday_years = list(range(2018, 2026))
    ca_holidays = holidays.CA(prov='ON', years=holiday_years)
    
    # df['is_holiday'] = df['datetime'].dt.date.isin(ca_holidays)
    # df['is_weekend'] = df['day_of_week'].isin([5, 6])\
    dates = pd.Series(datetime_index.date)
    day_of_week = pd.Series(datetime_index.dayofweek)
    df['is_holiday'] = dates.isin(ca_holidays)
    df['is_weekend'] = day_of_week.isin([5, 6])
    
    return df

def create_lag_features(df, lags=[24, 48, 168]):
    """Create lagged features for the target variable."""
    for col in df.columns:
        for lag in lags:
            df[f'{col}_lag_{lag}'] = df[col].shift(lag)
    
    df = df.dropna() # remove null values 
    
    return df

In [125]:
for dataset in datasets:
    print(dataset.shape)

(635832, 73)
(26332, 145)
(63456, 9)
(2483, 9)


In [123]:
for i in range(len(datasets)):
    datasets[i] = create_lag_features(datasets[i])
    datasets[i] = add_time_features(datasets[i])
    display(datasets[i])

# hourly_data = create_lag_features(hourly_data)
# daily_data = create_lag_features(daily_data)
# hourly_ontario_demand = create_lag_features(hourly_ontario_demand)
# daily_ontario_demand = create_lag_features(daily_ontario_demand)

# hourly_data = add_time_features(hourly_data)
# daily_data = add_time_features(daily_data)
# hourly_ontario_demand = add_time_features(hourly_ontario_demand)
# daily_ontario_demand = add_time_features(daily_ontario_demand)




C:\Users\tanis\AppData\Local\Temp\ipykernel_32280\2544228172.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['day_of_week'] = datetime_index.dayofweek
C:\Users\tanis\AppData\Local\Temp\ipykernel_32280\2544228172.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['month'] = datetime_index.month
C:\Users\tanis\AppData\Local\Temp\ipykernel_32280\2544228172.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = va

temperature_2m (°C)  cloud_cover (%)  \
datetime            region                                            
2018-01-02 16:00:00 Toronto                   -7.0               97   
                    West                     -15.6               20   
2018-01-02 17:00:00 Bruce                     -6.0               86   
                    East                      -6.0               96   
                    Essa                      -5.9               95   
...                                            ...              ...   
2025-04-04 23:00:00 Northwest                  0.1              100   
                    Ottawa                     1.9              100   
                    Southwest                  5.2              100   
                    Toronto                    5.0              100   
                    West                       5.4              100   

                               cloud_cover_low (%)  cloud_cover_mid (%)  \
datetime            region                                                
2018-01-02 16:00:00 Toronto                     97                    0   
                    West                        20                    0   
2018-01-02 17:00:00 Bruce                       74                    0   
                    East                        10                   15   
                    Essa                        95                   83   
...                                            ...                  ...   
2025-04-04 23:00:00 Northwest                   18                   99   
                    Ottawa                       0                    0   
                    Southwest                    0                   88   
                    Toronto                      0                  100   
                    West                         0                   57   

                               cloud_cover_high (%)  wind_speed_10m (km/h)  \
datetime            region                                                   
2018-01-02 16:00:00 Toronto                       1                   22.1   
                    West                          0                   30.0   
2018-01-02 17:00:00 Bruce                        46                   34.5   
                    East                         95                   30.1   
                    Essa                          0                   22.8   
...                                             ...                    ...   
2025-04-04 23:00:00 Northwest                    47                   12.4   
                    Ottawa                      100                    7.4   
                    Southwest                   100                   14.8   
                    Toronto                     100                    7.2   
                    West                        100                   19.1   

                               wind_speed_100m (km/h)  \
datetime            region                              
2018-01-02 16:00:00 Toronto                      35.8   
                    West                         52.4   
2018-01-02 17:00:00 Bruce                        49.8   
                    East                         47.9   
                    Essa                         37.6   
...                                               ...   
2025-04-04 23:00:00 Northwest                    24.6   
                    Ottawa                       15.0   
                    Southwest                    27.5   
                    Toronto                      17.6   
                    West                         35.8   

                               weather_code (wmo code)  precipitation (mm)  \
datetime            region                                                   
2018-01-02 16:00:00 Toronto                          3                 0.0   
                    West                             1                 0.0   
2018-01-02 17:00:00 Bruce                            3        

C:\Users\tanis\AppData\Local\Temp\ipykernel_32280\2544228172.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col}_lag_{lag}'] = df[col].shift(lag)
C:\Users\tanis\AppData\Local\Temp\ipykernel_32280\2544228172.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col}_lag_{lag}'] = df[col].shift(lag)
C:\Users\tanis\AppData\Local\Temp\ipykernel_32280\2544228172.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Con

temperature_2m_mean (°C)  cloud_cover_mean (%)  \
date       region                                                      
2018-01-18 Toronto                        -6.8                  55.0   
           West                           -6.1                  16.0   
2018-01-19 Bruce                          -0.7                  76.0   
           East                           -1.7                  86.0   
           Essa                           -2.6                  93.0   
...                                        ...                   ...   
2025-04-04 Northwest                       0.2                  82.0   
           Ottawa                          4.9                  49.0   
           Southwest                       5.1                  71.0   
           Toronto                         5.3                  67.0   
           West                            7.3                  82.0   

                      cloud_cover_max (%)  cloud_cover_min (%)  \
date       region                                                
2018-01-18 Toronto                  100.0                  2.0   
           West                      92.0                  0.0   
2018-01-19 Bruce                    100.0                  2.0   
           East                     100.0                 12.0   
           Essa                     100.0                  5.0   
...                                   ...                  ...   
2025-04-04 Northwest                100.0                  0.0   
           Ottawa                   100.0                  0.0   
           Southwest                100.0                  0.0   
           Toronto                  100.0                  0.0   
           West                     100.0                  0.0   

                      wind_speed_10m_mean (km/h)  temperature_2m_min (°C)  \
date       region                                                           
2018-01-18 Toronto                          14.7                    -10.6   
           West                             24.5                     -9.3   
2018-01-19 Bruce                            26.5                     -2.3   
           East                             15.9                     -5.0   
           Essa                             13.1                     -6.2   
...                                          ...                      ...   
2025-04-04 Northwest                        11.1                     -4.1   
           Ottawa                           11.7                     -0.5   
           Southwest                         8.3                     -0.8   
           Toronto                           7.1                     -0.3   
           West                             10.6                      1.9   

                      apparent_temperature_mean (°C)  weather_code (wmo code)  \
date       region                                                               
2018-01-18 Toronto                             -12.2                      3.0   
           West                                -13.0                      3.0   
2018-01-19 Bruce                                -7.3                      3.0   
           East                                 -6.7                     71.0   
           Essa                                 -7.3                     71.0   
...                                              ...                      ...   
2025-04-04 Northwest                            -4.2                     51.0   
           Ottawa                                1.1                      3.0   
           Southwest                             2.1                      3.0   
           Toronto                               2.5                      3.0   
           West                                  3.9                     51.0   

                      temperature_2m_max (°C)  apparent_temperature_max (°C)  \
date       region                                                              
2018-01-18

C:\Users\tanis\AppData\Local\Temp\ipykernel_32280\2544228172.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'{col}_lag_{lag}'] = df[col].shift(lag)
C:\Users\tanis\AppData\Local\Temp\ipykernel_32280\2544228172.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'{col}_lag_{lag}'] = df[col].shift(lag)
C:\Users\tanis\AppData\Local\Temp\ipykernel_32280\2544228172.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_index

,Ontario Demand,Ontario Demand_lag_24,Ontario Demand_lag_48,Ontario Demand_lag_168,day_of_week,month,day_of_year,is_holiday,is_weekend
datetime,,,,,,,,,
2018-01-08 00:00:00,15521,17015.0,17293.0,16627.0,0,1,8,NaN,NaN
2018-01-08 01:00:00,15129,16531.0,16937.0,16084.0,0,1,8,NaN,NaN
2018-01-08 02:00:00,14916,16240.0,16505.0,15866.0,0,1,8,NaN,NaN
2018-01-08 03:00:00,14880,16103.0,16599.0,15725.0,0,1,8,NaN,NaN
2018-01-08 04:00:00,14919,16031.0,16647.0,15470.0,0,1,8,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2025-04-04 19:00:00,16611,16951.0,18558.0,17732.0,4,4,94,NaN,NaN
2025-04-04 20:00:00,16209,16682.0,17765.0,17260.0,4,4,94,NaN,NaN
2025-04-04 21:00:00,15299,15739.0,16733.0,16358.0,4,4,94,NaN,NaN


C:\Users\tanis\AppData\Local\Temp\ipykernel_32280\2544228172.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['day_of_week'] = datetime_index.dayofweek
C:\Users\tanis\AppData\Local\Temp\ipykernel_32280\2544228172.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['month'] = datetime_index.month
C:\Users\tanis\AppData\Local\Temp\ipykernel_32280\2544228172.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = va

,Ontario Demand,Ontario Demand_lag_24,Ontario Demand_lag_48,Ontario Demand_lag_168,day_of_week,month,day_of_year,is_holiday,is_weekend
date,,,,,,,,,
2018-06-18,449568,368424.0,322065.0,406340.0,0,6,169,NaN,NaN
2018-06-19,386619,351924.0,333736.0,430873.0,1,6,170,NaN,NaN
2018-06-20,377181,348860.0,344165.0,434960.0,2,6,171,NaN,NaN
2018-06-21,365664,408330.0,318967.0,433198.0,3,6,172,NaN,NaN
2018-06-22,349240,405836.0,292545.0,455055.0,4,6,173,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2025-03-31,373406,406231.0,443595.0,333870.0,0,3,90,NaN,NaN
2025-04-01,375761,387399.0,454031.0,362343.0,1,4,91,NaN,NaN
2025-04-02,406097,370923.0,447541.0,366050.0,2,4,92,NaN,NaN


In [124]:
# we want to make sure that not too many rows have been dropped from each dataset and the number of columns increase by approx 3

for dataset in datasets:
    print(dataset.shape)

635832
26332
63456
2483
